In [39]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import random


In [28]:
try:

    import sys
    import os

    from fp.fp import FreeProxy
    from fake_useragent import UserAgent
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver import Chrome
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    import time
    print('all module are loaded ')

except Exception as e:

    print("Error ->>>: {} ".format(e))


class Spoofer(object):

    def __init__(self, country_id=['US'], rand=True, anonym=True):
        self.country_id = country_id
        self.rand = rand
        self.anonym = anonym
        self.userAgent, self.ip = self.get()

    def get(self):
        ua = UserAgent()
        proxy = FreeProxy(country_id=self.country_id, rand=self.rand, anonym=self.anonym).get()
        ip = proxy.split("://")[1]
        return ua.random, ip


class DriverOptions(object):

    def __init__(self):

        self.options = Options()
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--start-maximized')
        self.options.add_argument('--start-fullscreen')
        self.options.add_argument('--single-process')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument("--incognito")
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_experimental_option('useAutomationExtension', False)
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_argument("disable-infobars")

        self.helperSpoofer = Spoofer()

        self.options.add_argument('user-agent={}'.format(self.helperSpoofer.userAgent))
        self.options.add_argument('--proxy-server=%s' % self.helperSpoofer.ip)


class WebDriver(DriverOptions):

    def __init__(self, path=''):
        DriverOptions.__init__(self)
        self.driver_instance = self.get_driver()

    def get_driver(self):

        print("""
        IP:{}
        UserAgent: {}
        """.format(self.helperSpoofer.ip, self.helperSpoofer.userAgent))

        PROXY = self.helperSpoofer.ip
        webdriver.DesiredCapabilities.CHROME['proxy'] = {
            "httpProxy":PROXY,
            "ftpProxy":PROXY,
            "sslProxy":PROXY,
            "noProxy":None,
            "proxyType":"MANUAL",
            "autodetect":False
        }
        webdriver.DesiredCapabilities.CHROME['acceptSslCerts'] = True

        path = os.path.join(os.getcwd(), '/chromedriver.exe')

        driver = webdriver.Chrome()
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source":
                "const newProto = navigator.__proto__;"
                "delete newProto.webdriver;"
                "navigator.__proto__ = newProto;"
        })

        return driver

all module are loaded 


In [107]:
# Create anti bot detection web driver

flag = True
while flag:
    try:
        driver = WebDriver()
        flag = False
    except: # If Driver creation fails try again
        print("Driver creation failed.....Trying again")
        continue
print("Driver Successfully created")

driverinstance = driver.driver_instance
url = "https://www.expedia.com/Hotel-Search?adults=2&d1=2022-01-17&d2=2022-01-18&destination=Jaipur%2C%20Rajasthan%2C%20India&directFlights=false&endDate=2022-01-17&hotels-destination=Jaipur%2C%20Rajasthan%2C%20India&hotels-destination=Jaipur%2C%20Rajasthan%2C%20India&latLong=26.891643000499514%2C75.74652282695956&localDateFormat=M%2Fd%2Fyyyy&partialStay=false&regionId=1669&semdtl=&sort=RECOMMENDED&startDate=2022-01-15&theme=&useRewards=false&userIntent="
driverinstance.get(url)

# Click on the "SHOW MORE" button 20 times to load enough hotels 
for i in range(20):
    sleep_time = 5 + 5*random.random()
    time.sleep(sleep_time)
    driverinstance.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        button = driverinstance.find_element_by_xpath('//*[@class="uitk-button uitk-button-medium uitk-button-secondary"]')
        button.click()
    except:
        print("Button not Found")
        break

# Create BeautifulSoup instance and search for each hotel on the page to get it's link
soup = str(BeautifulSoup(driverinstance.page_source, 'html.parser'))
soup = BeautifulSoup(soup, 'lxml')

ol = soup.find('ol' , class_  = 'results-list no-bullet')
li = ol.find_all('a', class_='listing__link uitk-card-link')
hotel_url_list = []
li.pop(0)
for x,i in enumerate(li):
    hotel_url_list.append("https://www.expedia.com"+i.attrs['href'])
    print(x)

Driver creation failed.....Trying again

        IP:20.47.108.204:8888
        UserAgent: Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36
        
Driver Successfully created


KeyboardInterrupt: 

In [105]:
# Save all the links in a text file

with open("hotel_urls.txt", 'w') as output:
    for row in hotel_url_list:
        output.write(str(row) + '\n')

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.expedia.com/Hotel-Search?adults=2&d1=2022-01-10&d2=2022-01-11&destination=Mumbai%20%28and%20vicinity%29%2C%20Maharashtra%2C%20India&endDate=2022-01-11&latLong=19.107168%2C72.891596&regionId=6050062&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2022-01-10&theme=&useRewards=false&userIntent="
soup = BeautifulSoup(requests.get(url).content, "html.parser")

script = soup.select_one('[type="text/show-more"]')
script.replace_with(BeautifulSoup(script.contents[0], "html.parser"))

all_data = []
for item in soup.select(".fixresitem"):
    all_data.append(item.get_text(strip=True, separator="|").split("|")[:5])
    all_data[-1].append(
        item.findprevious(class="fixresheader2").get_text(strip=True)
    )

df = pd.DataFrame(
    all_data, columns=["Team 1", "Score 1", "Score 2", "Time", "Team 2", "Date"]
)
print(df)
df.to_csv("data.csv", index=False)